In [2]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1 # Given code seems to use tensorflow v1
import numpy as np
import time

from sklearn import preprocessing

# For easy testing (immediate evaluation)
tf.compat.v1.enable_eager_execution() 

# tf.compat.v1.disable_eager_execution()

### The 6 data files being trained from

**disp_coord**
- Discrete coordinates [0, 256-0] - [256, 256-0]

**disp_data**
- A bunch of float pairs (what does it represent?). 
- *Is it the displacement at each of the discrete coordinates in disp_coord?*
- *Based on it's usage in the provided code, seems to represent the axial displacement.*

**m_data**
- Young's Modulus? $256^2$ data points.
- Is this representing the $E$?

**nu_data**
- Poisson's Ration, $\nu$.
- $256^2$ data points.

**strain_coord**
- $256^2$ data points
- Discretized coordinates? [0.5, 255.5-0.5] - [255.5, 255.5-0.5]
- Likely the discretized coordinates for **strain data**

**strain_data**
- $256^2$ data points
- A bunch of float triplets. Could they represent:
  - $\epsilon_{xx}$
  - $\epsilon_{yy}$
  - $\gamma_{xy}$
- Which would then mean it represents the displacements
  - So would be used when positions are known, but both elasticity values (young's modulus and poisson's ratio) are unknown.



### Standardization

In [27]:
# Standardize features by removing the mean and scaling to unit variance.
# Think back to z-scores and standardization in PHYS 216
# The standard score of a sample x is calculated as:
#     z = (x - u) / s
# where u is the mean of the training samples or zero if with_mean=False, 
# and s is the standard deviation of the training samples
ss_x = preprocessing.StandardScaler()

x_disp = ss_x.fit_transform(np.asarray([i for i in range(11)]).reshape(-1, 1))
print(x_disp) # Standardized


[[-1.58113883]
 [-1.26491106]
 [-0.9486833 ]
 [-0.63245553]
 [-0.31622777]
 [ 0.        ]
 [ 0.31622777]
 [ 0.63245553]
 [ 0.9486833 ]
 [ 1.26491106]
 [ 1.58113883]]


### Building the Model

#### Initial Guesses

In [48]:
# Creates a random distribution based on a truncated normal with the 
# given standard deviation
tf1.truncated_normal([4, 4], stddev = 0.1) # Only in v1

<tf.Tensor 'truncated_normal:0' shape=(4, 4) dtype=float32>

In [35]:
# Self evident. Creates a tensor of given shape initialized to the given
# value (in this case 0.1)
tf.constant(0.1, shape = [4, 4])

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1]], dtype=float32)>

In [38]:
# Creates a variable version for tensor flow
tf.Variable(tf.constant(0.1, shape = [3,3]))

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1]], dtype=float32)>

#### Placeholder (what is the point?)

In [46]:
# Requires 
tf1.placeholder(tf.float32, [None, 2])

<tf.Tensor 'Placeholder_1:0' shape=(None, 2) dtype=float32>

#### Activation Functions

In [ ]:
#  https://en.wikipedia.org/wiki/Rectifier_(neural_networks)
# x if x > 0
# 0 otherwise
# Returns a tensor
tf.nn.relu()

In [ ]:
# https://medium.com/@akp83540/silu-sigmoid-linear-unit-activation-function-d9b6845f0c81 
# SiLU or swish activation function
# x * sigmoid(beta * x)
# Returns a tensor
tf.nn.swish()

In [ ]:
# Used in argument of activation function
# good old matrix multiplication. Used to multiply previous h (hidden layer?)
# with the current layers weights. (To which the current layers bias is added)
# QUESTION: Pretty sure the use of tf.Variable makes these (h, W, b) modifiable 
# by the training portion of the code
tf.matmul()

### Strain Calculation and Loss

#### Test tf math

The math part of the code uses the placeholders, I wonder if that is what enables
it's use during the training portion.

In [ ]:
thing = tf1.constant(0.3, dtype=tf.float32)

In [ ]:
tf.nn.conv2d()
tf.stack()

#### Error and Loss

In [13]:
# Reduces the dimensions of a tensor by reducing the provided
# dimension (default None, so all) to a single element equal to the mean
# in that dimension.
x = tf.constant([[2., 2.], [2., 2.], [0, 2]])
print(tf.reduce_mean(x)) # Average of all the cells

tf.Tensor(1.6666666, shape=(), dtype=float32)


### Training

In [51]:
session = tf1.Session()
session = tf.compat.v1.Session() # To read documentation

#### Where the Placeholders are used

From what I understand. This means that the placeholders (-s_----) are replaced
by there none placeholder equivalents (the imported data, -_----)

In [ ]:
# The optional feed_dict argument allows the caller to override the value of 
# tensors in the graph. Each key in feed_dict can be one of the following types:
#
# - If the key is a tf.Tensor, the value may be a Python scalar, string, list, 
#   or numpy ndarray that can be converted to the same dtype as that tensor. 
#   Additionally, if the key is a tf.compat.v1.placeholder, the shape of the 
#   value will be checked for compatibility with the placeholder.
# 
# - If the key is a tf.sparse.SparseTensor, the value should be a 
#   tf.compat.v1.SparseTensorValue.
# 
# - If the key is a nested tuple of Tensors or SparseTensors, the value should 
#   be a nested tuple with the same structure that maps to their corresponding 
#   values as above.
# 
# Each value in feed_dict must be convertible to a numpy array of the dtype of 
# the corresponding key.

session.run()